# 📘 Notebook: Modular Multi-Agent Architecture with LLMs

This notebook demonstrates how to structure and execute modular components using agents, orchestrators, and LLMs. 
It includes an example of how a user can interact with specialized agents through a defined module pipeline.


## 🔧 Setup Agent Framework Components

We import the core components needed to build and coordinate agents, including `User`, `Agent`, `Module`, and `Orchestrator`.


In [ ]:
from agents import User, Agent, Module, Orchestrator
import os

## ⚙️ LLM Configuration and Module Setup

This block defines the configuration for the LLM and sets up modules that the agent system will use. 
Modules can be chained, parallelized, or triggered based on the orchestration logic.


In [ ]:
# LLM configuration
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "gemma2-9b-it",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

# User agent
user = User(
    description="You are a human user looking for content suggestions such as books or movies."
)

# Agent that talks directly to the user
vendedor = Agent(
    name="vendedor",
    llm_config=llm_config,
    system_message="""
    You are a helpful assistant engaging in friendly conversation.
    Do not make any recommendations yourself.
""",
    description="Converses with the user"
)

# Tool: content recommender function
def get_content_recommendation(preference: str) -> str:
    """
    Returns a single content recommendation based on the user's preference.

    Parameters:
        preference (str): Either 'book' or 'movie'

    Returns:
        str: A recommended title
    """
    preference = preference.strip().lower()

    if preference == "book":
        return "Sapiens: A Brief History of Humankind – Yuval Noah Harari"
    elif preference == "movie":
        return "Inception – directed by Christopher Nolan"
    else:
        return "Please specify either 'book' or 'movie'."

# Agent: makes the recommendation
recommender_agent = Agent(
    name="recommender_agent",
    llm_config=llm_config,
    system_message="""
    You are a content recommendation agent.
    Your task is to recommend a "book" or "movie" based on the user request using the 'get_content_recommendation' tool.
""",
    description="Recommends a book or movie using a fixed tool.",
    tools=[get_content_recommendation],
    reflect_on_tool_use=True,
)

# Agent: explains the recommendation
explainer_agent = Agent(
    name="explainer_agent",
    llm_config=llm_config,
    system_message="""
    You are an explanation agent.
    Your task is to explain the recommendation made by the recommender agent. Just it.
""",
    description="Explains the recommendation",
)



allowed_speaker_transitions_dict = {
    recommender_agent: [explainer_agent],
}
# Module: internal logic of the recommendation
rec_module = Module(
    admin_name="content_module",
    agents=[recommender_agent, explainer_agent],
    speaker_selection_method="round_robin",
    allowed_or_disallowed_speaker_transitions=allowed_speaker_transitions_dict,
    speaker_transitions_type="allowed",
)

# Entry point to start the conversation flow
rec_orc = Orchestrator(
    name="rec_orchestrator",
    module=rec_module,
    llm_config=llm_config,
    system_message="Só repasse a mensagem.",
    description="Orchestrates the content recommendation process.",
)

main_module = Module(
    admin_name="main_module",
    agents=[user, rec_orc, vendedor],
    speaker_selection_method="auto",
    allowed_or_disallowed_speaker_transitions= {
        user:[vendedor, rec_orc],
        vendedor:[user],
        rec_orc:[user],
    },
    speaker_transitions_type="allowed",
)

# Orchestrator to manage the conversation flow
arara_orc = Orchestrator(
    name = "main_orchestrator",
    module=main_module,
    llm_config=llm_config,
    system_message="""
    You are the main orchestrator of the conversation.
""",
)

## 💬 Example Interaction Between User and Agent

This cell initiates a conversation between the user and a selected agent module. 
It shows how user inputs are processed and passed through the agent architecture.


In [ ]:
chat_result = user.talk_to(
    arara_orc, message="Me recomende um filme",
    cache=None
)